In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

def search_amazon_product(product):
    driver = webdriver.Chrome()  
    driver.get("https://www.amazon.in/")

    search_bar = driver.find_element_by_id("twotabsearchtextbox")
    search_bar.clear()
    search_bar.send_keys(product)
    search_bar.send_keys(Keys.RETURN)

    driver.implicitly_wait(5)

    products = driver.find_elements_by_xpath("//span[@class='a-size-medium a-color-base a-text-normal']")
    product_titles = [product.text for product in products]

    driver.quit()

    return product_titles

user_input = input("Enter the product to search for on Amazon India: ")
search_results = search_amazon_product(user_input)

if search_results:
    print("Search Results:")
    for i, result in enumerate(search_results, 1):
        print(f"{i}. {result}")
else:
    print("No products found for the given input.")


In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def scrape_product_details(product, num_pages):
    driver = webdriver.Chrome()  
    driver.get("https://www.amazon.in/")

    search_bar = driver.find_element_by_id("twotabsearchtextbox")
    search_bar.clear()
    search_bar.send_keys(product)
    search_bar.send_keys(Keys.RETURN)

    product_details = []

    for page in range(num_pages):
        try:
           
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//span[@class='a-size-medium a-color-base a-text-normal']")))
        
            products = driver.find_elements_by_xpath("//div[@data-component-type='s-search-result']")
            for product in products:
                try:
                    brand_name = product.find_element_by_xpath(".//span[@class='a-size-base-plus a-color-base']").text
                    product_name = product.find_element_by_xpath(".//span[@class='a-size-medium a-color-base a-text-normal']").text
                    price = product.find_element_by_xpath(".//span[@class='a-price-whole']").text
                    return_exchange = product.find_element_by_xpath(".//span[contains(text(),'Exchange') or contains(text(),'Returns')]//following-sibling::span").text
                    expected_delivery = product.find_element_by_xpath(".//span[contains(text(),'FREE Delivery')]//following-sibling::span").text
                    availability = product.find_element_by_xpath(".//span[contains(text(),'In stock')]//ancestor::div[@class='a-section a-spacing-none a-spacing-top-micro']").text
                    product_url = product.find_element_by_xpath(".//a[@class='a-link-normal a-text-normal']").get_attribute('href')

                    product_details.append({
                        "Brand Name": brand_name,
                        "Name of the Product": product_name,
                        "Price": price,
                        "Return/Exchange": return_exchange,
                        "Expected Delivery": expected_delivery,
                        "Availability": availability,
                        "Product URL": product_url
                    })
                except Exception as e:
                    print("Error scraping product details:", e)
                    continue
          
            next_button = driver.find_element_by_xpath("//li[@class='a-last']//a")
            if "Next" in next_button.text:
                next_button.click()
                time.sleep(2)  
            else:
                break
        except Exception as e:
            print("Error scraping page:", e)
            break

    driver.quit()

    return pd.DataFrame(product_details)

user_input = input("Enter the product to search for on Amazon India: ")
num_pages = 3  
product_details_df = scrape_product_details(user_input, num_pages)

product_details_df.fillna('-', inplace=True)

product_details_df.to_csv('amazon_product_details.csv', index=False)

print("Scraping and saving completed successfully.")


In [ ]:
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

def scrape_images(keyword, num_images):
    driver = webdriver.Chrome() 
    driver.get("https://www.google.com/imghp?hl=en")

    search_bar = driver.find_element_by_xpath("//input[@class='gLFyf gsfi']")
    search_bar.clear()
    search_bar.send_keys(keyword)
    search_bar.send_keys(Keys.RETURN)

    time.sleep(2)

    for _ in range(3):  
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    image_elements = driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']")
    image_urls = []
    for img in image_elements[:num_images]:
        try:
            img.click()
            time.sleep(1)
            actual_image = driver.find_element_by_css_selector('img.n3VNCb')
            if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                image_urls.append(actual_image.get_attribute('src'))
        except Exception as e:
            print("Error while scraping image URL:", e)
            continue

    driver.quit()

    return image_urls

keywords = ['fruits', 'cars', 'Machine Learning', 'Guitar', 'Cakes']
num_images_per_keyword = 10

for keyword in keywords:
    image_urls = scrape_images(keyword, num_images_per_keyword)
    save_directory = f'images/{keyword}'
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)
    for i, image_url in enumerate(image_urls):
        try:
            image_content = requests.get(image_url).content
            with open(os.path.join(save_directory, f'{keyword}_{i+1}.jpg'), 'wb') as f:
                f.write(image_content)
        except Exception as e:
            print("Error while saving image:", e)

print("Scraping and saving completed successfully.")


In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def scrape_smartphones(keyword):
    driver = webdriver.Chrome()  
    driver.get("https://www.flipkart.com/")

    search_bar = driver.find_element_by_xpath("//input[@title='Search for products, brands and more']")
    search_bar.clear()
    search_bar.send_keys(keyword)
    search_bar.send_keys(Keys.RETURN)

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[@class='_4rR01T']")))

    products = driver.find_elements_by_xpath("//div[@class='_4rR01T']")
    smartphone_details = []
    for product in products:
        try:
            brand_name = product.find_element_by_xpath(".//div[@class='_2WkVRV']").text
            smartphone_name = product.find_element_by_xpath(".//a[@class='s1Q9rs']").text
            color = product.find_element_by_xpath(".//a[@class='IRpwTa']").get_attribute('title')
            ram = product.find_element_by_xpath(".//li[contains(text(),'RAM')]").text
            storage = product.find_element_by_xpath(".//li[contains(text(),'ROM')]").text
            primary_camera = product.find_element_by_xpath(".//li[contains(text(),'Primary')]").text
            secondary_camera = product.find_element_by_xpath(".//li[contains(text(),'Secondary')]").text
            display_size = product.find_element_by_xpath(".//li[contains(text(),'Display')]").text
            battery_capacity = product.find_element_by_xpath(".//li[contains(text(),'Battery')]").text
            price = product.find_element_by_xpath(".//div[@class='_30jeq3']").text
            product_url = product.find_element_by_xpath(".//a[@class='s1Q9rs']").get_attribute('href')

            smartphone_details.append({
                "Brand Name": brand_name,
                "Smartphone Name": smartphone_name,
                "Colour": color,
                "RAM": ram,
                "Storage(ROM)": storage,
                "Primary Camera": primary_camera,
                "Secondary Camera": secondary_camera,
                "Display Size": display_size,
                "Battery Capacity": battery_capacity,
                "Price": price,
                "Product URL": product_url
            })
        except Exception as e:
            print("Error while scraping product details:", e)
            continue

    driver.quit()

    return pd.DataFrame(smartphone_details)

user_input = input("Enter the smartphone to search for on Flipkart: ")
smartphone_details_df = scrape_smartphones(user_input)

smartphone_details_df.fillna('-', inplace=True)

smartphone_details_df.to_csv('flipkart_smartphone_details.csv', index=False)

print("Scraping and saving completed successfully.")


In [ ]:
import selenium
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time

driver = selenium.webdriver.Chrome()

city = "New York City"
search_parameters = {"route": city}

driver.get("https://www.google.com/maps")

search_box = driver.find_element_by_name("q")
search_box.send_keys(city)
search_box.send_keys(Keys.RETURN)

wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".section-result")))

results = driver.find_elements_by_css_selector(".section-result")
coordinates = []
for result in results:
    latitude = result.find_element_by_css_selector(".latitude").text
    longitude = result.find_element_by_css_selector(".longitude").text
    coordinates.append((latitude, longitude))

print(coordinates)

driver.quit()

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_gaming_laptops():
    url = "https://www.digit.in/top-products/best-gaming-laptops-40.html"
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        laptops = soup.find_all('div', class_='TopNumbeHeading active sticky-footer')

        laptop_details = []
        for laptop in laptops:
            try:
                laptop_name = laptop.find('div', class_='heading-wraper').text.strip()
                specs = laptop.find_all('div', class_='specs-wraper')[0].find_all('div', class_='value')
                processor = specs[0].text.strip()
                memory = specs[1].text.strip()
                os = specs[2].text.strip()
                display = specs[3].text.strip()
                dimensions = specs[4].text.strip()
                weight = specs[5].text.strip()
                gpu = specs[6].text.strip()
                price = laptop.find('div', class_='smprice').text.strip()

                laptop_details.append({
                    "Laptop Name": laptop_name,
                    "Processor": processor,
                    "Memory": memory,
                    "Operating System": os,
                    "Display": display,
                    "Dimensions": dimensions,
                    "Weight": weight,
                    "Graphics Processor": gpu,
                    "Price": price
                })
            except Exception as e:
                print("Error scraping laptop details:", e)
                continue

        return pd.DataFrame(laptop_details)
    else:
        print("Failed to fetch page")
        return None

gaming_laptops_df = scrape_gaming_laptops()

if gaming_laptops_df is not None:
    gaming_laptops_df.to_csv('gaming_laptops_digit.csv', index=False)
    print("Scraping and saving completed successfully.")


In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://www.forbes.com/billionaires/?list=worlds-billionaires"

response = requests.get(url)

soup = BeautifulSoup(response.content, "html.parser")

billionaires = soup.find_all("tr", class_="profile-list__item")
for billionaire in billionaires:
    rank = billionaire.find("td", class_="profile-list__item-rank").text.strip()
    name = billionaire.find("td", class_="profile-list__item-name").text.strip()
    net_worth = billionaire.find("td", class_="profile-list__item-money").text.strip()
    age = billionaire.find("td", class_="profile-list__item-age")
    if age:
        age = age.text.strip()
    else:
        age = "N/A"
    citizenship = billionaire.find("td", class_="profile-list__item-citizenship").text.strip()
    source = billionaire.find("td", class_="profile-list__item-source")
    if source:
        source = source.text.strip()
    else:
        source = "N/A"
    industry = billionaire.find("td", class_="profile-list__item-industry").text.strip()
    print(f"Rank: {rank}")
    print(f"Name: {name}")
    print(f"Net worth: {net_worth}")
    print(f"Age: {age}")
    print(f"Citizenship: {citizenship}")
    print(f"Source: {source}")
    print(f"Industry: {industry}\n")

In [ ]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import os
import csv
from datetime import datetime


API_KEY = "YOUR_API_KEY"  
youtube = build('youtube', 'v3', developerKey=API_KEY)

def get_video_comments(video_id, max_results=500):
    try:
      
        comments = []
        next_page_token = None

        while len(comments) < max_results:
            response = youtube.commentThreads().list(
                part='snippet',
                videoId=video_id,
                maxResults=min(max_results - len(comments), 100),
                pageToken=next_page_token
            ).execute()

            for item in response['items']:
                comment = item['snippet']['topLevelComment']['snippet']
                comment_text = comment['textDisplay']
                comment_like_count = comment['likeCount']
                comment_published_time = comment['publishedAt']
                comment_published_time = datetime.strptime(comment_published_time, "%Y-%m-%dT%H:%M:%SZ")
                comments.append({
                    "Comment": comment_text,
                    "Like Count": comment_like_count,
                    "Published Time": comment_published_time
                })

            if 'nextPageToken' in response:
                next_page_token = response['nextPageToken']
            else:
                break

        return comments
    except HttpError as e:
        print(f"An HTTP error {e.resp.status} occurred: {e.content}")
        return []

def save_comments_to_csv(video_id, comments):
    filename = f"comments_{video_id}.csv"
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Comment', 'Like Count', 'Published Time']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for comment in comments:
            writer.writerow(comment)

    print(f"Comments saved to {filename}")

if __name__ == "__main__":
    video_id = input("Enter the YouTube video ID: ")
    comments = get_video_comments(video_id)
    if comments:
        save_comments_to_csv(video_id, comments)
    else:
        print("No comments found for the video.")


In [ ]:
pip install requests beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup

def scrape_hostels(location):
    url = f"https://www.hostelworld.com/findabed.php/ChosenCity.London/ChosenCountry.England"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        hostel_listings = soup.find_all("div", class_="hwta-property")
        
        hostels_data = []
        for hostel in hostel_listings:
            hostel_name = hostel.find("h2", class_="title").text.strip()
            distance_from_city_centre = hostel.find("span", class_="hwta-property-cta distance").text.strip()
            ratings = hostel.find("div", class_="score orange").text.strip()
            total_reviews = hostel.find("div", class_="reviews").text.strip()
            overall_reviews = hostel.find("div", class_="keyword").text.strip()
            privates_from_price = hostel.find("div", class_="prices").find("div", class_="private-prices").text.strip()
            dorms_from_price = hostel.find("div", class_="prices").find("div", class_="dorm-prices").text.strip()
            facilities = [facility.text.strip() for facility in hostel.find_all("div", class_="facilities")]
            property_description = hostel.find("div", class_="description").text.strip()
            
            hostels_data.append({
                "Hostel Name": hostel_name,
                "Distance from City Centre": distance_from_city_centre,
                "Ratings": ratings,
                "Total Reviews": total_reviews,
                "Overall Reviews": overall_reviews,
                "Privates From Price": privates_from_price,
                "Dorms From Price": dorms_from_price,
                "Facilities": facilities,
                "Property Description": property_description
            })
        
        return hostels_data
    else:
        print("Failed to fetch page")
        return None

london_hostels_data = scrape_hostels("London")

if london_hostels_data:
    print("Number of hostels found:", len(london_hostels_data))
    print("Example Hostel Data:")
    print(london_hostels_data[0])
